In [2]:
import sys
import os 
import nest_asyncio

# Sanity check
print(sys.executable)
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv() 

/Users/amorvan/Documents/code_dw/llm_collection/.venv/bin/python


True

In [3]:
import os
from pydantic import BaseModel, Field
from llama_index.core.workflow import (
    Workflow,
    step,
    Event,
    Context,
    StartEvent,
    StopEvent
)
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool


## 1 - How to create an agentic bot that answers based on the politeness of the user

2 LLM calls are used : 
- One to select among tools, the description allows to know which one to use
- Maybe a second call is used if the right tool is selected (respond polite)

  


In [ ]:


class AgenticWorkflow(Workflow):
    llm: OpenAI = OpenAI(model="gpt-4o-mini")

    @step
    async def chat(
        self, ev: StartEvent, 
    ) -> StopEvent:
        
        user_msg = ev["user_input"]

        async def run_impolite(query: str) -> str:
            """
            You should use this tool when the user is impolite 
            """
            return str("I only answer to nice people")

        async def run_polite(query: str) -> str:
            """Use this tool when the user is polite"""
            result = self.llm.complete(query)
            return "I like you. <3 " + result.text 

        impolite_tool = FunctionTool.from_defaults(async_fn=run_impolite)
        polite_tool = FunctionTool.from_defaults(async_fn=run_polite)

        # Based on the politeness
        response = await self.llm.apredict_and_call(
            [impolite_tool, polite_tool],
            user_msg=user_msg,
            error_on_no_tool_call=False,
        )

        return StopEvent(result=response)


In [ ]:
w = AgenticWorkflow(timeout=60)

r = await w.run(user_input="You are a dumb bot, tell me a joke")
r

In [ ]:
r = await w.run(user_input="Please tell me a joke")
r

## 2 - Exercise : Make a bot equipped with weather forecast tool

In [4]:


class AgenticWorkflow(Workflow):
    llm: OpenAI = OpenAI(model="gpt-4o-mini")

    @step
    async def chat(
        self, ev: StartEvent, 
    ) -> StopEvent:
        
        user_msg = ev["user_input"]

        async def weather_forecast(city: str) -> str:
            """
            This tool is used to give the weather forecast for a city
            """
            return str(f"it will be sunny in {city}")

        async def run_generic(query: str) -> str:
            """Use this tool to answer generic queries not realted to a weather forecast"""
            result = self.llm.complete(query)
            return result.text 

        weather_forecast_tool = FunctionTool.from_defaults(async_fn=weather_forecast)
        generic_tool = FunctionTool.from_defaults(async_fn=run_generic)

        # Based on the politeness
        response = await self.llm.apredict_and_call(
            [weather_forecast_tool, generic_tool],
            user_msg=user_msg,
            error_on_no_tool_call=False,
        )

        return StopEvent(result=response)


In [5]:
w = AgenticWorkflow(timeout=60)

r = await w.run(user_input="What is the weather in Paris ? ")
r

AgentChatResponse(response='it will be sunny in Paris', sources=[ToolOutput(content='it will be sunny in Paris', tool_name='weather_forecast', raw_input={'args': ('Paris',), 'kwargs': {}}, raw_output='it will be sunny in Paris', is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)